# 2023-2 언어데이터과학 24강 (2023-11-29) 실습 (1) `gensim` 패키지와 Word2Vec 모델 훈련

## 오늘의 목표

1. 모두의 말뭉치 '온라인 대화 말뭉치'의 발화 자료로 Word2Vec Skip-gram 모델을 훈련시킬 수 있다.
2. 단어 사이의 의미 유사성을 벡터들의 코사인 유사도로 계량화하여 설명할 수 있다.
3. t-SNE 기법을 사용하여 단어 벡터들을 2차원 평면에 시각화할 수 있다.
4. Word2Vec 모델을 사용하여 연령대에 따른 단어의 분포 변화를 추적할 수 있다.

## 0. 준비

### 파이썬 모듈 설치하기

Python에서 Word2Vec 모델을 사용하기 위해서는 `gensim` 모듈이 필요하다.

In [1]:
%pip install gensim

   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.2/24.0 MB 6.9 MB/s eta 0:00:04
    --------------------------------------- 0.3/24.0 MB 5.2 MB/s eta 0:00:05
    --------------------------------------- 0.6/24.0 MB 4.7 MB/s eta 0:00:06
   - -------------------------------------- 0.8/24.0 MB 4.1 MB/s eta 0:00:06
   - -------------------------------------- 0.9/24.0 MB 4.5 MB/s eta 0:00:06
   - -------------------------------------- 1.1/24.0 MB 4.2 MB/s eta 0:00:06
   - -------------------------------------- 1.2/24.0 MB 4.1 MB/s eta 0:00:06
   -- ------------------------------------- 1.3/24.0 MB 3.5 MB/s eta 0:00:07
   -- ------------------------------------- 1.3/24.0 MB 3.4 MB/s eta 0:00:07
   -- ------------------------------------- 1.4/24.0 MB 3.2 MB/s eta 0:00:08
   -- ------------------------------------- 1.5/24.0 MB 3.2 MB/s eta 0:00:08
   -- ------------------------------------- 1.6/24.0 MB 3.1 MB/s eta 0:00:08
   -- 

## 1. 데이터 가공

12–15강에서 만든 모두의 말뭉치 [온라인 대화 말뭉치] 파일을 읽고 `gensim` 모듈에서 사용 가능한 코퍼스로 가공하자.

In [42]:
import pandas as pd
from gensim.models import FastText,Word2Vec
from tqdm import tqdm

### 데이터 파일 읽기

In [2]:
DATA_PATH = '../data/NIKL_OM_form_age_sex.csv.tar.gz'

In [3]:
utterances = pd.read_csv(DATA_PATH, compression='gzip', on_bad_lines='skip')
utterances

,data/NIKL_OM_form_age_sex.csv,form,speaker_id,age,sex
0,MDRW2100000001.1.1,안녕하세요,MDRW2100000001_1,20대,여성
1,MDRW2100000001.1.4,이거 해봐요><,MDRW2100000001_1,20대,여성
2,MDRW2100000001.1.7,오 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,MDRW2100000001_1,20대,여성
3,MDRW2100000001.1.8,안챙겨도 잘커요,MDRW2100000001_1,20대,여성
4,MDRW2100000001.1.9,너무 맞는데요ㅜㅜ?,MDRW2100000001_1,20대,여성
...,...,...,...,...,...
2977836,MMRW2100000241.1.2775,한 번도 안 써봄...?,MMRW2100000241_2,20대,여성
2977837,MMRW2100000241.1.2776,그거 개꿀인디,MMRW2100000241_2,20대,여성
2977838,MMRW2100000241.1.2780,ㅋㅋㅋㅋㅋㅋ잠수복 개귀여웤ㅋㅋㅋㅋ,MMRW2100000241_2,20대,여성
2977839,MMRW2100000241.1.2786,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ가즈아!,MMRW2100000241_2,20대,여성


In [4]:
utterances = pd.read_csv(DATA_PATH, compression='gzip', on_bad_lines='skip')
utterances.dropna(inplace=True) #remove NAN
utterances.rename(columns={utterances.columns[0]: 'id'}, inplace=True)
utterances.set_index('id', inplace=True)
utterances

,form,speaker_id,age,sex
id,,,,
MDRW2100000001.1.1,안녕하세요,MDRW2100000001_1,20대,여성
MDRW2100000001.1.4,이거 해봐요><,MDRW2100000001_1,20대,여성
MDRW2100000001.1.7,오 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,MDRW2100000001_1,20대,여성
MDRW2100000001.1.8,안챙겨도 잘커요,MDRW2100000001_1,20대,여성
MDRW2100000001.1.9,너무 맞는데요ㅜㅜ?,MDRW2100000001_1,20대,여성
...,...,...,...,...
MMRW2100000241.1.2774,그 낚시대회 전용 투망 있을걸???,MMRW2100000241_2,20대,여성
MMRW2100000241.1.2775,한 번도 안 써봄...?,MMRW2100000241_2,20대,여성
MMRW2100000241.1.2776,그거 개꿀인디,MMRW2100000241_2,20대,여성


### 데이터 형변환

일반적으로 Python의 여러 라이브러리에서 코퍼스를 다룰 때는 한 문장을 단어들의 리스트로 표현하고, 코퍼스 전체를 문장들의 리스트로 표현한다.

지금 가지고 있는 데이터프레임에서는 문장에 해당하는 발화가 `str` 자료형이므로, `str.split()` 메소드를 사용하여 단어들의 리스트로 만들어 주자.

In [108]:
#corpus = utterances['form'].apply(lambda x: x.split())
corpus = utterances['form'].str.split()

print(corpus[:5])

id
MDRW2100000001.1.1            [안녕하세요]
MDRW2100000001.1.4        [이거, 해봐요><]
MDRW2100000001.1.7    [오, ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ]
MDRW2100000001.1.8        [안챙겨도, 잘커요]
MDRW2100000001.1.9      [너무, 맞는데요ㅜㅜ?]
Name: form, dtype: object


## 2. Word2Vec 모델 훈련

### 모델 초기화

In [144]:
model_ft = FastText(
    sg=1,
    min_count=5,
    vector_size=100,
    window=2,
    negative=5,
)

In [145]:
model_wv = Word2Vec(
    sg=1,
    min_count=5,
    vector_size=100,
    window=2,
    negative=5,
)

### Vocabulary 구축

In [146]:
model_wv.build_vocab(corpus_iterable=tqdm(corpus))
model_ft.build_vocab(corpus_iterable=tqdm(corpus))



  0%|          | 0/2977840 [00:00<?, ?it/s]

100%|██████████| 2977840/2977840 [00:04<00:00, 732373.65it/s]


### 모델 훈련시키기

In [166]:
model_ft.train(
    corpus_iterable=corpus,
    total_examples=model_ft.corpus_count,
    epochs=5
    )
# (trained_word_count, raw_word_count)

(32843946, 46433495)

In [167]:
model_wv.train(
    corpus_iterable=corpus,
    total_examples=model_wv.corpus_count,
    epochs=5
    )

(32846176, 46433495)

### 훈련 결과

Vocabulary 확인

In [168]:
# 빈도순 상위 30개 단어
print(model_ft.wv.index_to_key[:30])

['아', 'ㅋㅋ', 'ㅋㅋㅋ', '저는', '진짜', '근데', '너무', 'ㅎㅎ', '다', '저도', '좀', 'ㅋㅋㅋㅋ', '잘', '그', '나', '네', '더', '많이', '전', '그냥', '오', '난', 'ㅠㅠ', '맞아요', 'ㅋㅋㅋㅋㅋ', '안', '오늘', '저', '그래서', '그럼']


In [169]:
print(model_wv.wv.index_to_key[:30])

['아', 'ㅋㅋ', 'ㅋㅋㅋ', '저는', '진짜', '근데', '너무', 'ㅎㅎ', '다', '저도', '좀', 'ㅋㅋㅋㅋ', '잘', '그', '나', '네', '더', '많이', '전', '그냥', '오', '난', 'ㅠㅠ', '맞아요', 'ㅋㅋㅋㅋㅋ', '안', '오늘', '저', '그래서', '그럼']


단어 벡터 확인

In [170]:
print(model_ft.wv['ㅋㅋ'])

[-0.4131657  -0.39832586 -0.4931168   0.46783248  0.05336297  0.58905673
  0.07295868  0.02378595 -0.35117695  0.694327   -0.08639729  0.11334926
  0.32361615 -0.35400665 -0.39117372 -0.2478193   0.11739188  0.32393304
 -0.22600591  0.29527676 -0.11225966 -0.30208057  0.23438776  0.12904209
  0.3850736  -0.31787986  0.07234293  0.17055392  0.02818342  0.49705926
 -0.24286045  0.03747735 -0.1906997  -0.24348928  0.2773308  -0.18998289
 -0.41618276 -0.04307976 -0.0594615   0.4543349  -0.13232957 -0.11233735
 -0.05504943 -0.05118585  0.09832597 -0.15176275 -0.03553076  0.18323626
  0.5643102   0.07187416 -0.28856754 -0.41442072  0.12958662 -0.21614516
  0.31491506  0.07101652  0.26358724  0.51581085 -0.30619782  0.5256351
 -0.26580116  0.22447717 -0.0432996   0.06143727 -0.15347473 -0.88791823
  0.31791466  0.08579479  0.5605376   0.08042997 -0.24256048 -0.19718724
  0.1745815   0.03932945 -0.08996779  0.16967073  0.05092776  0.45784178
 -0.560753   -0.13546142 -0.17827076  0.19341874 -0.

In [171]:
print(model_wv.wv['ㅋㅋ'])

[ 5.2550864e-01 -2.6849902e-01  1.2246246e-01  1.6852971e-02
 -7.1183163e-01  1.0029305e-01  1.5794741e-02  7.5923401e-01
 -5.3677171e-01 -1.8677065e-01  1.7028482e-01 -7.8881055e-02
 -2.5271568e-01  2.2267970e-01  5.5723060e-02 -1.5085919e-01
 -2.2906829e-03  6.2930234e-02 -4.9478136e-02 -1.5095969e-01
  4.4127387e-01 -3.2216895e-01  4.2074567e-01 -6.8183976e-01
 -3.7522547e-02 -3.7996131e-01  7.1610457e-01 -3.1716067e-02
 -1.2834917e-01  6.1624670e-01 -2.7134230e-02 -3.5353369e-01
  4.2295402e-01 -6.8841763e-02  1.4173739e-01 -3.2330629e-01
  2.5463554e-01 -1.8211189e-01  2.9046756e-01 -4.9836838e-01
  1.6846625e-02 -5.1230586e-01 -7.6364875e-02 -6.4626920e-01
  5.9527129e-02 -1.5894307e-01 -5.6245494e-01 -4.8842841e-01
  5.1087815e-01  5.1128685e-01  3.2288200e-01 -7.9752809e-01
  3.5263810e-02 -6.8209255e-01 -2.9637524e-01  1.3189164e-01
  1.3422504e-01  4.4881716e-01 -3.7882057e-01  3.3934796e-01
 -9.1383256e-02 -4.4309455e-01  2.3012069e-01 -3.2693785e-01
 -4.9648464e-01 -2.75169

## 3. Word2Vec 모델 활용

### `gensim`의 주요 기능



코사인 유사도 계산

In [174]:
print(model_wv.wv.similarity(w1='펜', w2='연필'))
print(model_wv.wv.similarity(w1='펜', w2='좋아요'))
print(model_ft.wv.similarity(w1='펜', w2='연필'))
print(model_ft.wv.similarity(w1='펜', w2='좋아요'))


0.657671
0.18230473
0.6293597
0.22218569


평행사변형 모형

In [175]:
# 아빠 : 엄마 = X : 할머니
# 아빠 - 엄마 = X - 할머니
# X = 할머니 + 아빠 - 엄마

model_ft.wv.most_similar(positive=['아빠','할머니'], negative=['엄마'], topn=10)

# X =/ "할아버지"

[('할머니나', 0.8292530179023743),
 ('할머니들', 0.8241649866104126),
 ('할머니와', 0.8185047507286072),
 ('할머니만', 0.8109624981880188),
 ('할머니댁', 0.8095919489860535),
 ('할머니의', 0.8057341575622559),
 ('할머니를', 0.8002930283546448),
 ('할머니댁에', 0.786259651184082),
 ('할머니네', 0.7842994928359985),
 ('할머니집', 0.7811053395271301)]

### '완전'과 '아주'의 이웃 비교

In [176]:
model_ft.wv.most_similar(['완전'])

[('나완전', 0.8251630067825317),
 ('완전완전', 0.8149611949920654),
 ('완전ㅠ', 0.7941586375236511),
 ('완전!', 0.7569344639778137),
 ('완전요', 0.6449913382530212),
 ('완전체', 0.6365385055541992),
 ('완전좋아', 0.6315881013870239),
 ('완전!!', 0.5789616107940674),
 ('완전체로', 0.5673215389251709),
 ('짱', 0.5666264295578003)]

In [177]:
model_wv.wv.most_similar(['완전'])

[('짱', 0.5487399697303772),
 ('아주', 0.5431596636772156),
 ('진짜', 0.5305799245834351),
 ('대박', 0.5019885897636414),
 ('장난', 0.4931302070617676),
 ('정말', 0.47955238819122314),
 ('진심', 0.47520413994789124),
 ('진쨔', 0.45907124876976013),
 ('비주얼', 0.4584898054599762),
 ('핵', 0.4560844302177429)]

In [178]:
model_ft.wv.most_similar(['아주'])

[('아주아주', 0.7857614755630493),
 ('아주~', 0.7349845767021179),
 ('아주대', 0.6601337194442749),
 ('아주조아', 0.6247730851173401),
 ('아주좋아', 0.6087311506271362),
 ('아주조금', 0.5997405052185059),
 ('정말정말', 0.5931103229522705),
 ('아주그냥', 0.5825241804122925),
 ('아주가끔', 0.5765763521194458),
 ('아주많이', 0.573191225528717)]

In [179]:
model_wv.wv.most_similar(['아주'])

[('매우', 0.6038323640823364),
 ('정말', 0.5912904739379883),
 ('굉장히', 0.5641878843307495),
 ('무척', 0.5525595545768738),
 ('정말정말', 0.5467406511306763),
 ('완전', 0.5431596040725708),
 ('아주아주', 0.5387831926345825),
 ('상당히', 0.5364410877227783),
 ('마냥', 0.5196735262870789),
 ('만들어줘서', 0.4966734051704407)]

In [180]:
model_ft.wv.most_similar(['^^'])

[('^^^', 0.860858142375946),
 ('^^^^', 0.8360406160354614),
 ('^^7', 0.7935317158699036),
 ('ㅎㅎ^^', 0.7874269485473633),
 ('..^^', 0.7871010303497314),
 ('~~^^', 0.7739741206169128),
 ('~~~^^', 0.7726141214370728),
 ('아^^', 0.7526466846466064),
 ('^^!', 0.7503291368484497),
 ('^^~', 0.7417525053024292)]

In [181]:
model_wv.wv.most_similar(['^^'])

[(':)', 0.7448242902755737),
 ('ㅎㅎㅎㅎ', 0.7445195913314819),
 ('~', 0.738932192325592),
 ('ㅎㅎㅎ', 0.7042863368988037),
 ('~~', 0.7032327651977539),
 ('~!!', 0.7011941075325012),
 ('ㅎ.ㅎ', 0.6938683390617371),
 ('!', 0.689638078212738),
 ('><', 0.6885297894477844),
 ('ㅎㅎ', 0.685219407081604)]

In [182]:
model_ft.wv.most_similar(['ㅋㅋ'])

[('ㅋㅋㅋ', 0.9154864549636841),
 ('ㅋㅋㅋㄱㄱㅋㅋ', 0.8514170050621033),
 ('ㅋㅋㅋㅌㅋㅋ', 0.8447259664535522),
 ('ㅋㅋㅋㄲㅋㅋ', 0.8420471549034119),
 ('ㅋㅋㅋ엌ㅋㅋ', 0.8403575420379639),
 ('ㅋㅋㅌㅋㅋ', 0.8315430879592896),
 ('ㅋㅋㅋ케', 0.8304737210273743),
 ('ㅋㅋㄲㅋㅋ', 0.8258742690086365),
 ('ㅋㅋㄱㄱㅋㅋ', 0.8258450627326965),
 ('ㅋㅋㅋㄴ', 0.8250867128372192)]

In [183]:
model_wv.wv.most_similar(['ㅋㅋ'])

[('ㅋㅋㅋ', 0.9297182559967041),
 ('ㅋㅋㅋㅋ', 0.868186354637146),
 ('ㅋㅋㅋㅋㅋ', 0.7790327072143555),
 ('ㅎㅎㅎ', 0.7761170864105225),
 ('ㅋㅋㅋㅋㅋㅋ', 0.774032473564148),
 ('ㅋㅋㄱ', 0.7341235876083374),
 ('ㅋㅋㅋㅋㅋㅋㅋ', 0.7040610909461975),
 ('ㅋㅋㅋㅋㅋㅋㅋㅋㅋ', 0.7008540630340576),
 ('ㅎㅎ', 0.698562502861023),
 ('ㅋㅋㅋㅋㅋㅋㅋㅋ', 0.6952998042106628)]

In [184]:
model_ft.wv.most_similar(['너뮤'])

[('너어무', 0.8217867016792297),
 ('느무', 0.8193405866622925),
 ('넘모', 0.8077669143676758),
 ('너무너무너무', 0.8074190616607666),
 ('너무...너무', 0.8053817749023438),
 ('상상만해도', 0.8018584847450256),
 ('너무너무너무너무', 0.7986644506454468),
 ('너무달아', 0.7929858565330505),
 ('너모', 0.788438618183136),
 ('너무귀엽고', 0.7883802652359009)]

In [185]:
model_wv.wv.most_similar(['너뮤'])

[('상상만해도', 0.8205316662788391),
 ('너무..', 0.7961815595626831),
 ('너모', 0.7917347550392151),
 ('너무너무너무', 0.7808246612548828),
 ('느무', 0.7785001993179321),
 ('넘넘', 0.7781725525856018),
 ('생각만해도', 0.7743363976478577),
 ('죽겠네', 0.773689866065979),
 ('넘모', 0.7725625038146973),
 ('ㅋㅋㅋ넘', 0.7703806161880493)]

FastText 모델은 단어의 형태적 특성을 포착해서 그 형태가 비슷한 단어들을 output으로 내보내고, Word2Vec은 의미적 특성을 similarity에 훨씬 더 많이 반영 시키는듯 합니다.